In [ ]:
%run -i rename.py

In [ ]:
from tqdm import tqdm
import torch
from torchvision.io import read_video, write_video
video, audio, metadata = read_video('F:/animation/t1.mkv')
video.shape, video.dtype

In [ ]:
from config import config
val_range = 255.
inp = video.permute(0, 3, 1, 2).to(dtype=config.dtype())

In [ ]:
from IFRNet import doSlomo, getOpt
from progress import Node
from imageProcess import extend
import os
import os.path as osp
os.chdir(osp.join(osp.dirname(osp.abspath(__file__)), '../'))
opt = getOpt({'model': 'M', 'sf': 2.0})
opt.start = 0
identity = lambda x=None, *_: x
p = doSlomo(identity, Node({'slomo': 'IFRNet'}, learn=0), opt)

In [ ]:
res = []
for frame in tqdm(inp):
  extend(res, p(frame.to(device=config.device()) / val_range))
extend(res, p(None))
res = [(x * val_range).clamp_(0, val_range).cpu() for x in res]
res = torch.stack(res)
video = res.to(dtype=torch.uint8, device=torch.device('cpu')).permute(0, 2, 3, 1)
video.shape

In [ ]:
from IFRNet_Model import Model as IFRNet
modelPath = './IFRNet_Vimeo90K.pth'
m = IFRNet()
m.load_state_dict(torch.load(modelPath, map_location='cpu'))
for param in m.parameters():
  param.requires_grad_(False)
m.eval()
m = m.to(dtype=config.dtype(), device=config.device())
embt = torch.tensor(1/2).view(1, 1, 1, 1).to(dtype=config.dtype(), device=config.device())

In [ ]:
from imageProcess import alignF, padImageReflect
oriHeight, oriWidth = video.shape[1], video.shape[2]
width, height = alignF[16](oriWidth), alignF[16](oriHeight)
pad = padImageReflect((0, width - oriWidth, 0, height - oriHeight))
unpad = lambda im: im[:, :oriHeight, :oriWidth]
oriHeight, height

In [ ]:
res = [inp[0]]
inp0 = res[-1].to(device=config.device()) / val_range
for inp1 in tqdm(inp[1:]):
  inpIn = inp1.to(device=config.device()) / val_range
  x = m.inference(pad(inp0.unsqueeze(0)), pad(inpIn.unsqueeze(0)), embt).squeeze(0)
  output = (unpad(x) * val_range).clamp_(0, val_range).cpu()
  res.append(output)
  res.append(inp1)
  inp0 = inpIn
video = torch.stack(res).permute(0, 2, 3, 1).to(dtype=torch.uint8)
video.shape, x.mean(), x.abs().max(), (x > 1).sum() / x.nelement()

In [ ]:
os.chdir(osp.dirname(osp.abspath(__file__)))
write_video('./download/target_m.mp4',
  video_array=video, fps=metadata['video_fps'] * 2, audio_array=audio, audio_fps=metadata['audio_fps'], audio_codec='aac',
  video_codec='hevc_nvenc',
  options={'pix_fmt':'p010le', 'rc':'vbr', 'cq':'26', 'preset:v':'p7', 'profile:v': 'main', 'tune': 'hq', 'tier': 'high', 'bf': '5', 'b_ref_mode':'middle', 'spatial_aq':'1', 'temporal-aq': '1', 'nonref_p': '1', 'rc-lookahead': '53', 'keyint_min':'1'})